# Content-based models for predition of Mood of audio


https://www.researchgate.net/figure/Two-dimensional-valence-arousal-space_fig1_304124018

https://cvml.unige.ch/databases/DEAM/

https://cvml.unige.ch/databases/emoMusic/

https://multimediaeval.github.io/editions/2021/tasks/music/


https://www.music-ir.org/mirex/wiki/2020:Audio_Classification_(Train/Test)_Tasks

https://www.music-ir.org/nema_out/mirex2020/results/act/mood_report/index.html

https://ismir2008.ismir.net/papers/ISMIR2008_263.pdf



https://developer.nvidia.com/cuda-downloads


### Essentia SVM models

Install gaia

    git clone git@github.com:MTG/gaia.git
    cd gaia
    # configure, but put it in our current virtual env
    ./waf configure --prefix=`pwd`/../env
    ./waf && ./waf install
    
Install essentia with gaia

Currently using https://github.com/MTG/essentia/pull/811 to use libswresample on ubuntu 20.10.
    
    git clone git@github.com:MTG/essentia.git
    cd essentia
    PKG_CONFIG_PATH=`pwd`/../env/lib/pkgconfig python3 waf configure --prefix=`pwd`/../env --with-gaia --with-example=streaming_extractor_music
    python waf && python waf install

Models

    wget https://essentia.upf.edu/svm_models/essentia-extractor-svm_models-v2.1_beta5.tar.gz
    tar xvfz essentia-extractor-svm_models-v2.1_beta5.tar.gz


Configuration of profile file
https://essentia.upf.edu/streaming_extractor_music.html

```
startTime: 0
outputFrames: 0
outputFormat: json
requireMbid: false
indent: 4

highlevel:
    compute: 1
    svm_models: ['essentia-extractor-svm_models-v2.1_beta5/mood_acoustic.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_aggressive.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_electronic.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_happy.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_party.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_relaxed.history',
		'essentia-extractor-svm_models-v2.1_beta5/mood_sad.history',
		'essentia-extractor-svm_models-v2.1_beta5/moods_mirex.history'
		]
```

Because we installed into a virtualenv, we need to set LD_LIBRARY_PATH:

    LD_LIBRARY_PATH=env/lib ./essentia/build/src/examples/essentia_streaming_extractor_music file.flac data.json profile

In [1]:
import json
import os
import subprocess

import requests
import numpy as np
import tabulate
from IPython.core.display import display, HTML

modeldir = "tfmodels"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [2]:
def compute_svm(audiofile, extractor, profilename):
    result = subprocess.run([extractor, audiofile, 'extractor_output.json', profilename], env={"LD_LIBRARY_PATH": os.path.join(os.getcwd(), "env", "lib")})
    # check result
    output = json.load(open('extractor_output.json'))
    
    os.unlink('extractor_output.json')
    
    result = {}
    for classifier, values in output["highlevel"].items():
        prob = round(values['probability'], 2)
        val = values['value']
        result[classifier] = f"{val} ({prob})"
    return result
    

### Essentia Tensorflow models

These are models trained on the same data sources as the SVM models, except using transfer learning on pre-generated _embeddings_.

Deep learning models have two important features, the dataset used to train the model, and the _architecture_, the design of the neural layers and how they are connected to each other.

For each classifier we have 5 different combinations of these values:

    dataset     msd      mtt     audioset
    embeddings  vgg      vgg
                musicnn  musicnn
                                 vggish
                                 
msd: Million Song Dataset (30 second audio samples)
mtt: Magnatagatune
musicnn: https://github.com/jordipons/musicnn
vgg: mhttps://neurohive.io/en/popular-networks/vgg16/
audioset & vggish: https://research.google.com/audioset/download.html


A list of these precomputed models is available at https://essentia.upf.edu/models.html#transfer-learning-classifiers

In [6]:
import requests
import os

modeldir = "tfmodels"

def download_model(model, arch, version, modeldir):
    fname = f"{model}-{arch}-{version}.pb"
    destpath = os.path.join(modeldir, fname)
    url = f"https://essentia.upf.edu/models/classifiers/{model}/{fname}"
    r = requests.get(url)
    r.raise_for_status()
    with open(destpath, "wb") as fp:
        fp.write(r.content)

for model in ["mood_acoustic", "mood_aggressive", "mood_electronic", "mood_happy", "mood_party", "mood_relaxed", "mood_sad", "moods_mirex"]:
    for arch in ["musicnn-msd", "musicnn-mtt", "vgg-msd", "vgg-mtt", "vggish-audioset"]:
        print(f"{model}-{arch}")
        version = "2"
        if arch == "vggish-audioset":
            version = "1"
        else:
            version = "2"
        os.makedirs(os.path.join(modeldir, model), exist_ok=True)
        fname = f"{model}-{arch}-{version}.pb"
        destpath = os.path.join(modeldir, model, fname)
        if not os.path.exists(destpath):
            try:
                download_model(model, arch, version, os.path.join(modeldir, model))
            except requests.HTTPError:
                # Couldn't find this file, try again with version 1
                try:
                    print("  - getting version 1")
                    download_model(model, arch, "1", os.path.join(modeldir, model))
                except requests.HTTPError:
                    # still not here, ignore
                    print(f"Cannot find {model}-{arch} version 2 or 1")


mood_acoustic-musicnn-msd
mood_acoustic-musicnn-mtt
mood_acoustic-vgg-msd
mood_acoustic-vgg-mtt
mood_acoustic-vggish-audioset
mood_aggressive-musicnn-msd
mood_aggressive-musicnn-mtt
mood_aggressive-vgg-msd
mood_aggressive-vgg-mtt
mood_aggressive-vggish-audioset
mood_electronic-musicnn-msd
mood_electronic-musicnn-mtt
mood_electronic-vgg-msd
mood_electronic-vgg-mtt
mood_electronic-vggish-audioset
mood_happy-musicnn-msd
mood_happy-musicnn-mtt
mood_happy-vgg-msd
mood_happy-vgg-mtt
mood_happy-vggish-audioset
mood_party-musicnn-msd
mood_party-musicnn-mtt
mood_party-vgg-msd
mood_party-vgg-mtt
mood_party-vggish-audioset
mood_relaxed-musicnn-msd
mood_relaxed-musicnn-mtt
mood_relaxed-vgg-msd
mood_relaxed-vgg-mtt
mood_relaxed-vggish-audioset
mood_sad-musicnn-msd
mood_sad-musicnn-mtt
mood_sad-vgg-msd
mood_sad-vgg-mtt
mood_sad-vggish-audioset
moods_mirex-musicnn-msd
  - getting version 1
moods_mirex-musicnn-mtt
  - getting version 1
moods_mirex-vgg-msd
  - getting version 1
moods_mirex-vgg-mtt
  - 

In [7]:
def compute_features(filepath, modelpath, modelname, modelarch):
    """Given a path to a model file and a path to an audio file, compute
    predictions."""
    audio = EasyLoader(filename=filepath, sampleRate=16000, endTime=180)()
    if "vggish" in modelarch:
        model = TensorflowPredictVGGish(graphFilename=modelpath)
    else:
        model = TensorflowPredictMusiCNN(graphFilename=modelpath)
    activations = model(audio)
    averaged_predictions = np.mean(activations, axis=0)
    return averaged_predictions

def get_model_file(modeldir, model, arch):
    """Find the model file for a given model + architecture.
    Most models have a v2, but some only have v1. Try and get the v2 one first
    and only get the v1 one if it doesn't exist.
    """
    if arch == "vggish-audioset":
        version = "1"
    else:
        version = "2"
    fname = f"{model}-{arch}-{version}.pb"
    destpath = os.path.join(modeldir, model, fname)
    if not os.path.exists(destpath):
        print(" - cannot find, trying v1")
        version = "1"
        fname = f"{model}-{arch}-{version}.pb"
        destpath = os.path.join(modeldir, model, fname)
        if not os.path.exists(destpath):
            print(" - still cannot find, skipping")
            return None
    return destpath

def compute_all_tf_models_for_file(audiofile):
    results = {}
    # , "moods_mirex"
    for model in ["mood_acoustic", "mood_aggressive", "mood_electronic", "mood_happy", "mood_party", "mood_relaxed", "mood_sad"]:
        for arch in ["musicnn-msd", "musicnn-mtt", "vgg-msd", "vgg-mtt", "vggish-audioset"]:
            print(f"{model}-{arch}")
            modelfile = get_model_file(modeldir, model, arch)
            if modelfile:
                predictions = compute_features(audiofile, modelfile, model, arch)
                top_class = np.argmax(predictions)
                cls = classifier_labels[model][top_class]
                value = round(predictions[top_class].item(), 2)
                model_label = model + "-" + arch
                results[model_label] = f"{cls} ({round(value, 2)})"
                #results[model_label] = (cls, value)
    return results

In [8]:
j = json.load(open("metadata_and_spotify.json"))

In [9]:
from essentia.standard import EasyLoader, TensorflowPredictMusiCNN, TensorflowPredictVGGish
import essentia
essentia.log.infoActive = False
essentia.log.warningActive = False

mirex_moods = [
"passionate, rousing, confident, boisterous, rowdy",
"rollicking, cheerful, fun, sweet, amiable/good natured",
"literate, poignant, wistful, bittersweet, autumnal, brooding",
"humorous, silly, campy, quirky, whimsical, witty, wry",
"aggressive, fiery, tense/anxious, intense, volatile, visceral"
]

classifier_labels = {
    "mood_acoustic": ["Acoustic", "Not acoustic"],
    "mood_aggressive": ["Aggressive", "Not aggressive"],
    "mood_electronic": ["Electronic", "Not electronic"],
    "mood_happy": ["Happy", "Not happy"],
    "mood_party": ["Not party", "Party"],
    "mood_relaxed": ["Not relaxed", "Relaxed"],
    "mood_sad": ["Not sad", "Sad"],
    "moods_mirex": ["class1", "class2", "class3", "class4", "class5"],
}

album_results = {}
album = j[-1]
root = album['root']
for file in album['files']:
    print("*", file)
    tf_result = compute_all_tf_models_for_file(os.path.join(root, file))
    album_results[file] = tf_result
    svm_result = compute_svm(os.path.join(root, file), './essentia/build/src/examples/essentia_streaming_extractor_music', 'profile')
    album_results[file].update(svm_result)
    

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


* 01 - Queen - Bohemian Rhapsody.mp3
mood_acoustic-musicnn-msd


2022-02-18 17:37:44.214375: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 17:37:44.214395: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-02-18 17:37:44.214942: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 17:37:44.214974: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


mood_acoustic-musicnn-mtt
mood_acoustic-vgg-msd
mood_acoustic-vgg-mtt
mood_acoustic-vggish-audioset
mood_aggressive-musicnn-msd
mood_aggressive-musicnn-mtt
mood_aggressive-vgg-msd
mood_aggressive-vgg-mtt
mood_aggressive-vggish-audioset
mood_electronic-musicnn-msd
mood_electronic-musicnn-mtt
mood_electronic-vgg-msd
mood_electronic-vgg-mtt
mood_electronic-vggish-audioset
mood_happy-musicnn-msd
mood_happy-musicnn-mtt
mood_happy-vgg-msd
mood_happy-vgg-mtt
mood_happy-vggish-audioset
mood_party-musicnn-msd
mood_party-musicnn-mtt
mood_party-vgg-msd
mood_party-vgg-mtt
mood_party-vggish-audioset
mood_relaxed-musicnn-msd
mood_relaxed-musicnn-mtt
mood_relaxed-vgg-msd
mood_relaxed-vgg-mtt
mood_relaxed-vggish-audioset
mood_sad-musicnn-msd
mood_sad-musicnn-mtt
mood_sad-vgg-msd
mood_sad-vgg-mtt
mood_sad-vggish-audioset


[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_acoustic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_aggressive.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_electronic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_happy.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_party.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_relaxed.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_sad.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/moods_mirex.history
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and

...

[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_acoustic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_aggressive.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_electronic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_happy.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_party.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_relaxed.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_sad.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/moods_mirex.history
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and

* 16 - Queen - We Will Rock You.mp3
mood_acoustic-musicnn-msd
mood_acoustic-musicnn-mtt
mood_acoustic-vgg-msd
mood_acoustic-vgg-mtt
mood_acoustic-vggish-audioset
mood_aggressive-musicnn-msd
mood_aggressive-musicnn-mtt
mood_aggressive-vgg-msd
mood_aggressive-vgg-mtt
mood_aggressive-vggish-audioset
mood_electronic-musicnn-msd
mood_electronic-musicnn-mtt
mood_electronic-vgg-msd
mood_electronic-vgg-mtt
mood_electronic-vggish-audioset
mood_happy-musicnn-msd
mood_happy-musicnn-mtt
mood_happy-vgg-msd
mood_happy-vgg-mtt
mood_happy-vggish-audioset
mood_party-musicnn-msd
mood_party-musicnn-mtt
mood_party-vgg-msd
mood_party-vgg-mtt
mood_party-vggish-audioset
mood_relaxed-musicnn-msd
mood_relaxed-musicnn-mtt
mood_relaxed-vgg-msd
mood_relaxed-vgg-mtt
mood_relaxed-vggish-audioset
mood_sad-musicnn-msd
mood_sad-musicnn-mtt
mood_sad-vgg-msd
mood_sad-vgg-mtt
mood_sad-vggish-audioset


[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_acoustic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_aggressive.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_electronic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_happy.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_party.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_relaxed.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_sad.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/moods_mirex.history
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and

* 17 - Queen - We Are the Champions.mp3
mood_acoustic-musicnn-msd
mood_acoustic-musicnn-mtt
mood_acoustic-vgg-msd
mood_acoustic-vgg-mtt
mood_acoustic-vggish-audioset
mood_aggressive-musicnn-msd
mood_aggressive-musicnn-mtt
mood_aggressive-vgg-msd
mood_aggressive-vgg-mtt
mood_aggressive-vggish-audioset
mood_electronic-musicnn-msd
mood_electronic-musicnn-mtt
mood_electronic-vgg-msd
mood_electronic-vgg-mtt
mood_electronic-vggish-audioset
mood_happy-musicnn-msd
mood_happy-musicnn-mtt
mood_happy-vgg-msd
mood_happy-vgg-mtt
mood_happy-vggish-audioset
mood_party-musicnn-msd
mood_party-musicnn-mtt
mood_party-vgg-msd
mood_party-vgg-mtt
mood_party-vggish-audioset
mood_relaxed-musicnn-msd
mood_relaxed-musicnn-mtt
mood_relaxed-vgg-msd
mood_relaxed-vgg-mtt
mood_relaxed-vggish-audioset
mood_sad-musicnn-msd
mood_sad-musicnn-mtt
mood_sad-vgg-msd
mood_sad-vgg-mtt
mood_sad-vggish-audioset


[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_acoustic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_aggressive.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_electronic.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_happy.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_party.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_relaxed.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/mood_sad.history
[   INFO   ] MusicExtractorSVM: adding SVM model essentia-extractor-svm_models-v2.1_beta5/moods_mirex.history
[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and

### Spotify data

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

valence (float)

A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
    
    
energy (float)

Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

In [10]:
for model in ["mood_acoustic", "mood_aggressive", "mood_electronic", "mood_happy", "mood_party", "mood_relaxed", "mood_sad"]:
    table = []
    archs = ["musicnn-msd", "musicnn-mtt", "vgg-msd", "vgg-mtt", "vggish-audioset"]
    table.append(["file"] + [model + "-" + a for a in archs] + [model])
    for file, data in album_results.items():
        table.append([file] + [data.get(model + "-" + a, '-') for a in archs] + [data[model]])
    tablehtml = tabulate.tabulate(table, tablefmt='html')
    display(HTML(f'<h3>{model}</h3>'))
    display(tablehtml)

file,mood_acoustic-musicnn-msd,mood_acoustic-musicnn-mtt,mood_acoustic-vgg-msd,mood_acoustic-vgg-mtt,mood_acoustic-vggish-audioset,mood_acoustic
01 - Queen - Bohemian Rhapsody.mp3,Not acoustic (0.85),Not acoustic (0.43),Acoustic (0.86),Not acoustic (0.6),Acoustic (0.6),not_acoustic (0.98)
02 - Queen - Another One Bites the Dust.mp3,Not acoustic (0.99),Not acoustic (0.86),Not acoustic (0.81),Not acoustic (0.83),Not acoustic (0.92),not_acoustic (0.98)
03 - Queen - Killer Queen.mp3,Not acoustic (0.61),Acoustic (0.44),Acoustic (0.78),Not acoustic (0.63),Not acoustic (0.68),not_acoustic (0.78)
04 - Queen - Fat Bottomed Girls.mp3,Not acoustic (0.97),Not acoustic (0.69),Acoustic (0.72),Not acoustic (0.71),Not acoustic (0.87),not_acoustic (0.98)
05 - Queen - Bicycle Race.mp3,Not acoustic (0.86),Not acoustic (0.57),Acoustic (0.65),Not acoustic (0.68),Not acoustic (0.81),not_acoustic (0.99)
06 - Queen - You’re My Best Friend.mp3,Not acoustic (0.85),Not acoustic (0.55),Acoustic (0.74),Not acoustic (0.65),Not acoustic (0.79),not_acoustic (0.95)
07 - Queen - Don’t Stop Me Now.mp3,Not acoustic (0.93),Not acoustic (0.62),Acoustic (0.56),Not acoustic (0.69),Not acoustic (0.81),not_acoustic (0.98)
08 - Queen - Save Me.mp3,Not acoustic (0.71),Acoustic (0.56),Acoustic (0.89),Acoustic (0.69),Acoustic (0.51),not_acoustic (0.74)
09 - Queen - Crazy Little Thing Called Love.mp3,Not acoustic (0.6),Not acoustic (0.75),Not acoustic (0.55),Not acoustic (0.71),Not acoustic (0.57),not_acoustic (0.86)
10 - Queen - Somebody to Love.mp3,Not acoustic (0.91),Not acoustic (0.63),Not acoustic (0.54),Not acoustic (0.78),Not acoustic (0.75),not_acoustic (0.95)


file,mood_aggressive-musicnn-msd,mood_aggressive-musicnn-mtt,mood_aggressive-vgg-msd,mood_aggressive-vgg-mtt,mood_aggressive-vggish-audioset,mood_aggressive
01 - Queen - Bohemian Rhapsody.mp3,Not aggressive (0.96),Not aggressive (0.89),Not aggressive (0.73),Not aggressive (0.66),Not aggressive (0.94),not_aggressive (1.0)
02 - Queen - Another One Bites the Dust.mp3,Not aggressive (0.91),Not aggressive (0.69),Aggressive (0.82),Aggressive (0.67),Not aggressive (0.78),not_aggressive (0.94)
03 - Queen - Killer Queen.mp3,Not aggressive (0.97),Not aggressive (0.9),Not aggressive (0.68),Aggressive (0.6),Not aggressive (0.86),not_aggressive (0.88)
04 - Queen - Fat Bottomed Girls.mp3,Not aggressive (0.92),Not aggressive (0.58),Aggressive (0.55),Aggressive (0.67),Aggressive (0.52),aggressive (0.59)
05 - Queen - Bicycle Race.mp3,Not aggressive (0.86),Not aggressive (0.65),Aggressive (0.59),Aggressive (0.69),Not aggressive (0.66),aggressive (0.77)
06 - Queen - You’re My Best Friend.mp3,Not aggressive (0.97),Not aggressive (0.89),Not aggressive (0.5),Aggressive (0.56),Not aggressive (0.8),aggressive (1.0)
07 - Queen - Don’t Stop Me Now.mp3,Not aggressive (0.89),Not aggressive (0.66),Aggressive (0.62),Aggressive (0.71),Not aggressive (0.61),not_aggressive (0.72)
08 - Queen - Save Me.mp3,Not aggressive (0.97),Not aggressive (0.95),Not aggressive (0.8),Not aggressive (0.74),Not aggressive (0.98),not_aggressive (1.0)
09 - Queen - Crazy Little Thing Called Love.mp3,Not aggressive (0.96),Not aggressive (0.81),Not aggressive (0.59),Aggressive (0.61),Not aggressive (0.83),not_aggressive (0.92)
10 - Queen - Somebody to Love.mp3,Not aggressive (0.9),Not aggressive (0.7),Aggressive (0.6),Aggressive (0.64),Not aggressive (0.74),aggressive (1.0)


file,mood_electronic-musicnn-msd,mood_electronic-musicnn-mtt,mood_electronic-vgg-msd,mood_electronic-vgg-mtt,mood_electronic-vggish-audioset,mood_electronic
01 - Queen - Bohemian Rhapsody.mp3,Not electronic (0.73),Not electronic (0.52),Not electronic (0.56),Electronic (0.51),Not electronic (0.64),not_electronic (0.72)
02 - Queen - Another One Bites the Dust.mp3,Electronic (0.55),Electronic (0.58),Electronic (0.84),Electronic (0.66),Electronic (0.84),electronic (0.81)
03 - Queen - Killer Queen.mp3,Not electronic (0.73),Not electronic (0.66),Electronic (0.49),Not electronic (0.65),Not electronic (0.63),not_electronic (0.74)
04 - Queen - Fat Bottomed Girls.mp3,Not electronic (0.68),Not electronic (0.66),Electronic (0.67),Not electronic (0.73),Not electronic (0.73),not_electronic (0.72)
05 - Queen - Bicycle Race.mp3,Not electronic (0.7),Not electronic (0.53),Electronic (0.54),Not electronic (0.63),Not electronic (0.58),electronic (0.54)
06 - Queen - You’re My Best Friend.mp3,Not electronic (0.79),Not electronic (0.59),Electronic (0.51),Not electronic (0.6),Not electronic (0.73),electronic (0.88)
07 - Queen - Don’t Stop Me Now.mp3,Not electronic (0.7),Not electronic (0.65),Not electronic (0.57),Not electronic (0.76),Not electronic (0.72),not_electronic (0.79)
08 - Queen - Save Me.mp3,Not electronic (0.83),Not electronic (0.75),Not electronic (0.6),Not electronic (0.8),Not electronic (0.78),not_electronic (0.93)
09 - Queen - Crazy Little Thing Called Love.mp3,Electronic (0.52),Electronic (0.5),Electronic (0.75),Electronic (0.59),Not electronic (0.58),not_electronic (0.63)
10 - Queen - Somebody to Love.mp3,Not electronic (0.62),Not electronic (0.52),Electronic (0.68),Electronic (0.51),Not electronic (0.54),electronic (0.88)


file,mood_happy-musicnn-msd,mood_happy-musicnn-mtt,mood_happy-vgg-msd,mood_happy-vgg-mtt,mood_happy-vggish-audioset,mood_happy
01 - Queen - Bohemian Rhapsody.mp3,Not happy (0.76),Not happy (0.75),Not happy (0.75),Not happy (0.69),Not happy (0.7),not_happy (0.62)
02 - Queen - Another One Bites the Dust.mp3,Happy (0.62),Happy (0.55),Not happy (0.67),Not happy (0.54),Not happy (0.61),not_happy (0.87)
03 - Queen - Killer Queen.mp3,Happy (0.7),Happy (0.64),Happy (0.62),Happy (0.65),Happy (0.68),happy (0.64)
04 - Queen - Fat Bottomed Girls.mp3,Not happy (0.62),Not happy (0.6),Not happy (0.62),Not happy (0.69),Happy (0.53),happy (0.67)
05 - Queen - Bicycle Race.mp3,Happy (0.6),Happy (0.49),Not happy (0.47),Happy (0.56),Happy (0.65),not_happy (0.59)
06 - Queen - You’re My Best Friend.mp3,Happy (0.74),Happy (0.62),Happy (0.59),Happy (0.52),Happy (0.65),not_happy (0.64)
07 - Queen - Don’t Stop Me Now.mp3,Happy (0.65),Happy (0.58),Happy (0.57),Happy (0.62),Happy (0.72),happy (0.6)
08 - Queen - Save Me.mp3,Not happy (0.63),Not happy (0.59),Not happy (0.65),Not happy (0.64),Not happy (0.69),happy (0.56)
09 - Queen - Crazy Little Thing Called Love.mp3,Happy (0.59),Not happy (0.57),Not happy (0.51),Happy (0.5),Not happy (0.6),not_happy (0.52)
10 - Queen - Somebody to Love.mp3,Happy (0.63),Happy (0.51),Not happy (0.48),Happy (0.52),Happy (0.59),happy (0.59)


file,mood_party-musicnn-msd,mood_party-musicnn-mtt,mood_party-vgg-msd,mood_party-vgg-mtt,mood_party-vggish-audioset,mood_party
01 - Queen - Bohemian Rhapsody.mp3,Not party (0.92),Not party (0.92),Not party (0.89),Not party (0.58),Not party (0.9),not_party (0.96)
02 - Queen - Another One Bites the Dust.mp3,Not party (0.76),Not party (0.56),Party (0.75),Party (0.73),Party (0.7),party (0.79)
03 - Queen - Killer Queen.mp3,Not party (0.86),Not party (0.69),Not party (0.76),Party (0.7),Not party (0.64),not_party (0.69)
04 - Queen - Fat Bottomed Girls.mp3,Not party (0.82),Not party (0.77),Not party (0.84),Not party (0.52),Not party (0.65),not_party (0.66)
05 - Queen - Bicycle Race.mp3,Not party (0.81),Not party (0.69),Not party (0.68),Party (0.7),Not party (0.51),party (0.87)
06 - Queen - You’re My Best Friend.mp3,Not party (0.85),Not party (0.74),Not party (0.85),Party (0.65),Not party (0.63),not_party (0.51)
07 - Queen - Don’t Stop Me Now.mp3,Not party (0.72),Not party (0.57),Not party (0.62),Party (0.64),Party (0.51),party (0.66)
08 - Queen - Save Me.mp3,Not party (0.93),Not party (0.9),Not party (0.92),Not party (0.68),Not party (0.88),not_party (0.87)
09 - Queen - Crazy Little Thing Called Love.mp3,Not party (0.89),Not party (0.82),Not party (0.53),Party (0.65),Not party (0.76),not_party (0.92)
10 - Queen - Somebody to Love.mp3,Not party (0.77),Not party (0.69),Not party (0.64),Party (0.69),Not party (0.6),party (0.65)


file,mood_relaxed-musicnn-msd,mood_relaxed-musicnn-mtt,mood_relaxed-vgg-msd,mood_relaxed-vgg-mtt,mood_relaxed-vggish-audioset,mood_relaxed
01 - Queen - Bohemian Rhapsody.mp3,Relaxed (0.86),Relaxed (0.77),Relaxed (0.77),Relaxed (0.79),Relaxed (0.84),relaxed (0.86)
02 - Queen - Another One Bites the Dust.mp3,Relaxed (0.54),Not relaxed (0.54),Not relaxed (0.7),Not relaxed (0.7),Not relaxed (0.7),not_relaxed (0.56)
03 - Queen - Killer Queen.mp3,Relaxed (0.53),Not relaxed (0.66),Not relaxed (0.51),Not relaxed (0.71),Not relaxed (0.53),relaxed (0.79)
04 - Queen - Fat Bottomed Girls.mp3,Relaxed (0.64),Not relaxed (0.61),Relaxed (0.56),Not relaxed (0.6),Not relaxed (0.81),not_relaxed (0.84)
05 - Queen - Bicycle Race.mp3,Relaxed (0.64),Not relaxed (0.61),Not relaxed (0.6),Not relaxed (0.78),Not relaxed (0.69),not_relaxed (0.72)
06 - Queen - You’re My Best Friend.mp3,Relaxed (0.52),Not relaxed (0.66),Relaxed (0.5),Not relaxed (0.66),Not relaxed (0.69),relaxed (0.95)
07 - Queen - Don’t Stop Me Now.mp3,Not relaxed (0.48),Not relaxed (0.78),Not relaxed (0.73),Not relaxed (0.81),Not relaxed (0.77),not_relaxed (0.68)
08 - Queen - Save Me.mp3,Relaxed (0.77),Relaxed (0.74),Relaxed (0.79),Relaxed (0.69),Relaxed (0.77),not_relaxed (0.55)
09 - Queen - Crazy Little Thing Called Love.mp3,Relaxed (0.66),Relaxed (0.63),Not relaxed (0.51),Not relaxed (0.6),Relaxed (0.67),not_relaxed (0.68)
10 - Queen - Somebody to Love.mp3,Relaxed (0.58),Not relaxed (0.65),Not relaxed (0.58),Not relaxed (0.71),Not relaxed (0.55),relaxed (0.93)


file,mood_sad-musicnn-msd,mood_sad-musicnn-mtt,mood_sad-vgg-msd,mood_sad-vgg-mtt,mood_sad-vggish-audioset,mood_sad
01 - Queen - Bohemian Rhapsody.mp3,Sad (0.91),Sad (0.52),Sad (0.73),Sad (0.64),Sad (0.59),not_sad (0.74)
02 - Queen - Another One Bites the Dust.mp3,Sad (0.68),Not sad (0.54),Not sad (0.94),Not sad (0.9),Not sad (0.92),not_sad (0.83)
03 - Queen - Killer Queen.mp3,Sad (0.75),Sad (0.45),Not sad (0.68),Not sad (0.62),Not sad (0.61),not_sad (0.77)
04 - Queen - Fat Bottomed Girls.mp3,Sad (0.73),Not sad (0.53),Not sad (0.57),Not sad (0.52),Not sad (0.8),not_sad (0.9)
05 - Queen - Bicycle Race.mp3,Sad (0.65),Not sad (0.5),Not sad (0.72),Not sad (0.79),Not sad (0.77),not_sad (0.98)
06 - Queen - You’re My Best Friend.mp3,Sad (0.71),Sad (0.49),Not sad (0.62),Not sad (0.57),Not sad (0.64),sad (0.99)
07 - Queen - Don’t Stop Me Now.mp3,Not sad (0.64),Not sad (0.53),Not sad (0.79),Not sad (0.77),Not sad (0.78),not_sad (0.94)
08 - Queen - Save Me.mp3,Sad (0.94),Sad (0.68),Sad (0.78),Sad (0.84),Sad (0.81),sad (0.59)
09 - Queen - Crazy Little Thing Called Love.mp3,Sad (0.78),Not sad (0.48),Not sad (0.75),Not sad (0.76),Not sad (0.76),not_sad (0.84)
10 - Queen - Somebody to Love.mp3,Sad (0.66),Not sad (0.48),Not sad (0.8),Not sad (0.8),Not sad (0.74),sad (0.96)
